In [14]:
!pip install transformers pandas

from transformers import TapasTokenizer, TapasForQuestionAnswering
import pandas as pd
import torch

def load_csv(file_path):
    return pd.read_csv(file_path)

def format_data_for_tapas(data):
    data = data.fillna("").astype(str)
    return data

def load_tapas_pipeline():
    tokenizer = TapasTokenizer.from_pretrained("google/tapas-large-finetuned-wtq")
    model = TapasForQuestionAnswering.from_pretrained("google/tapas-large-finetuned-wtq")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    print(f"Using device: {device}")
    return tokenizer, model, device

def answer_question(question, data, tokenizer, model, device):
    try:
        data = format_data_for_tapas(data)
        inputs = tokenizer(table=data, queries=[question], padding="max_length", return_tensors="pt")
        inputs = {key: value.to(device) for key, value in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits.cpu()
        predicted_answer_coordinates = tokenizer.convert_logits_to_predictions(
            {key: value.cpu() for key, value in inputs.items()}, logits
        )[0]
        cell_coords = predicted_answer_coordinates[0]
        if cell_coords:
            row, column = cell_coords[0]
            return data.iloc[row, column]
        else:
            return "No answer found."
    except Exception as e:
        return f"Error processing question: {e}"

if __name__ == "__main__":
    print("Upload your key-value CSV file.")

    from google.colab import files
    uploaded = files.upload()

    file_name = list(uploaded.keys())[0]

    data = load_csv(file_name)
    print("\nData loaded successfully. Here is the dataset:\n")
    data = format_data_for_tapas(data)
    print(data)

    tokenizer, model, device = load_tapas_pipeline()

    while True:
        question = input("Enter your question (or type 'exit' to quit): ")
        if question.lower() == 'exit':
            print("Exiting.")
            break
        try:
            answer = answer_question(question, data, tokenizer, model, device)
            print(f"Answer: {answer}\n")
        except Exception as e:
            print(f"Error: {e}\nPlease try again with a different question.")


Upload your key-value CSV file.


Saving fullWingDataTable.csv to fullWingDataTable (1).csv

Data loaded successfully. Here is the dataset:

             xc    yb surf section            cp
0   0.797794118   0.0    U       1  -0.126649077
1   0.738970588   0.0    U       1  -0.182058047
2   0.676470588   0.0    U       1   -0.22823219
3   0.619485294   0.0    U       1  -0.255936675
4   0.556985294   0.0    U       1  -0.283641161
5   0.498161765   0.0    U       1  -0.265171504
6   1.003577818  0.13    U       2   0.230211055
7   0.951699463  0.13    U       2    0.19347666
8   0.919499106  0.13    U       2   0.112337523
9   0.595706619  0.13    U       2   -0.30522398
10  0.559928444  0.13    U       2  -0.314815916
11  0.516994633  0.13    U       2  -0.306633146
12  0.466905188  0.13    U       2  -0.352286921
13  0.352415027  0.13    U       2  -0.309579584
14  0.293381038  0.13    U       2  -0.346442085
15  0.234347048  0.13    U       2  -0.383304586
16  1.001788909  0.13    L       2   0.265984655
17  0.95348

KeyboardInterrupt: Interrupted by user